<a href="https://colab.research.google.com/github/Gon-Frecces/ML-zoomcamp/blob/main/stvscurly_hair.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
import pandas as pd
import numpy as np


In [35]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [36]:
import tensorflow as tf
from tensorflow import keras

In [37]:
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import ImageDataGenerator




In [38]:
train_dir = '/content/drive/MyDrive/data/train'
test_dir =  '/content/drive/MyDrive/data/test'



In [39]:
from torchvision import transforms


train_transforms = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    ) # ImageNet normalization
])

In [40]:
from torchvision.datasets import ImageFolder

train_dataset = ImageFolder(
    root=train_dir,
    transform=train_transforms
)

validation_dataset = ImageFolder(
    root=test_dir,
    transform=train_transforms
)


In [41]:
from torch.utils.data import DataLoader

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
validation_loader = DataLoader(validation_dataset, batch_size=32, shuffle=False)


In [42]:
import numpy as np
import torch

SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)

if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [43]:
import torch.nn as nn
import torch.nn.functional as F

class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()

        # Convolutional layer
        self.conv1 = nn.Conv2d(
            in_channels=3,
            out_channels=32,
            kernel_size=(3, 3),
            stride=1,
            padding=0
        )

        # Max pooling
        self.pool = nn.MaxPool2d(kernel_size=(2, 2))

        # Compute flattened size:
        # Input: (3, 200, 200)
        # After Conv (3x3, no padding): (32, 198, 198)
        # After MaxPool (2x2): (32, 99, 99)
        self.fc1 = nn.Linear(32 * 99 * 99, 64)

        # Output layer
        self.fc2 = nn.Linear(64, 1)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)

        x = torch.flatten(x, start_dim=1)

        x = F.relu(self.fc1(x))

        # Binary classification output
        x = torch.sigmoid(self.fc2(x))
        return x


In [44]:
model = CNNModel()


In [45]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)


In [46]:
optimizer = torch.optim.SGD(
    model.parameters(),
    lr=0.002,
    momentum=0.8
)


In [47]:
criterion = nn.BCELoss()


In [48]:
dummy_input = torch.randn(1, 3, 200, 200).to(device)
output = model(dummy_input)
print(output.shape)

torch.Size([1, 1])


torch.Size([1, 1])


torch.Size([1, 1])

In [49]:
# Option 1: Using torchsummary (install with: pip install torchsummary)
from torchsummary import summary
summary(model, input_size=(3, 200, 200))

# Option 2: Manual counting
total_params = sum(p.numel() for p in model.parameters())
print(f"Total parameters: {total_params}")

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 198, 198]             896
         MaxPool2d-2           [-1, 32, 99, 99]               0
            Linear-3                   [-1, 64]      20,072,512
            Linear-4                    [-1, 1]              65
Total params: 20,073,473
Trainable params: 20,073,473
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.46
Forward/backward pass size (MB): 11.96
Params size (MB): 76.57
Estimated Total Size (MB): 89.00
----------------------------------------------------------------
Total parameters: 20073473


In [50]:
transforms.RandomRotation(50),
transforms.RandomResizedCrop(200, scale=(0.9, 1.0), ratio=(0.9, 1.1)),
transforms.RandomHorizontalFlip(),

(RandomHorizontalFlip(p=0.5),)

In [51]:
num_epochs = 20
history = {'acc': [], 'loss': [], 'val_acc': [], 'val_loss': []}

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        labels = labels.float().unsqueeze(1) # Ensure labels are float and have shape (batch_size, 1)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        # For binary classification with BCEWithLogitsLoss, apply sigmoid to outputs before thresholding for accuracy
        predicted = (torch.sigmoid(outputs) > 0.5).float()
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_dataset)
    epoch_acc = correct_train / total_train
    history['loss'].append(epoch_loss)
    history['acc'].append(epoch_acc)

    model.eval()
    val_running_loss = 0.0
    correct_val = 0
    total_val = 0
    with torch.no_grad():
        for images, labels in validation_loader:
            images, labels = images.to(device), labels.to(device)
            labels = labels.float().unsqueeze(1)

            outputs = model(images)
            loss = criterion(outputs, labels)

            val_running_loss += loss.item() * images.size(0)
            predicted = (torch.sigmoid(outputs) > 0.5).float()
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    val_epoch_loss = val_running_loss / len(validation_dataset)
    val_epoch_acc = correct_val / total_val
    history['val_loss'].append(val_epoch_loss)
    history['val_acc'].append(val_epoch_acc)

    print(f"Epoch {epoch+1}/{num_epochs}, "
          f"Loss: {epoch_loss:.4f}, Acc: {epoch_acc:.4f}, "
          f"Val Loss: {val_epoch_loss:.4f}, Val Acc: {val_epoch_acc:.4f}")

Epoch 1/20, Loss: 0.6483, Acc: 0.4888, Val Loss: 0.6150, Val Acc: 0.4876
Epoch 2/20, Loss: 0.5578, Acc: 0.4888, Val Loss: 0.6558, Val Acc: 0.4876
Epoch 3/20, Loss: 0.5094, Acc: 0.4888, Val Loss: 0.7056, Val Acc: 0.4876
Epoch 4/20, Loss: 0.4592, Acc: 0.4888, Val Loss: 0.8935, Val Acc: 0.4876
Epoch 5/20, Loss: 0.4811, Acc: 0.4888, Val Loss: 0.6638, Val Acc: 0.4876
Epoch 6/20, Loss: 0.4025, Acc: 0.4888, Val Loss: 0.6842, Val Acc: 0.4876
Epoch 7/20, Loss: 0.3436, Acc: 0.4888, Val Loss: 1.2078, Val Acc: 0.4876
Epoch 8/20, Loss: 0.5482, Acc: 0.4888, Val Loss: 0.6467, Val Acc: 0.4876
Epoch 9/20, Loss: 0.3481, Acc: 0.4888, Val Loss: 0.8384, Val Acc: 0.4876
Epoch 10/20, Loss: 0.2908, Acc: 0.4888, Val Loss: 0.9403, Val Acc: 0.4876
Epoch 11/20, Loss: 0.3127, Acc: 0.4888, Val Loss: 0.6595, Val Acc: 0.4876
Epoch 12/20, Loss: 0.2083, Acc: 0.4888, Val Loss: 0.8819, Val Acc: 0.4876
Epoch 13/20, Loss: 0.1703, Acc: 0.4888, Val Loss: 1.1547, Val Acc: 0.4876
Epoch 14/20, Loss: 0.1812, Acc: 0.4888, Val Los

In [53]:
median_acc = np.median(history['acc'])
std_loss = np.std(history['loss'])
print(median_acc)
print(std_loss)


mean_test_loss = np.mean(history['val_loss'])
print("Mean test loss (all epochs):", mean_test_loss)

avg_test_acc_last_5 = np.mean(history['val_acc'][5:10])
print("Average test accuracy (epochs 6–10):", avg_test_acc_last_5)


0.48877805486284287
0.18214966571835195
Mean test loss (all epochs): 0.8937009737933455
Average test accuracy (epochs 6–10): 0.4875621890547263
